In [1]:
import cv2
import numpy as np
from PIL import Image
import pytesseract
#import easyocr
from shapely.geometry import Point, Polygon

In [2]:
print(cv2.__version__)
print(cv2.__path__)

4.5.4
['c:\\Anaconda\\lib\\site-packages\\cv2']


In [3]:
circle_color = 'red'
overview_color = 'red'
minibox_color = 'blue'
color_dict = {'red': {'Lower': np.array([140, 240, 225]), 'Upper': np.array([170, 255, 245])},
              'blue': {'Lower': np.array([100, 80, 46]), 'Upper': np.array([124, 255, 255])},
              'green': {'Lower': np.array([35, 43, 35]), 'Upper': np.array([90, 255, 255])},
              }

In [4]:
def convert_coord(x, y, cvs_left, cvs_top, sx, sy, sw, sh, dx, dy, dw, dh):
    # print(x, y, cvs_left, cvs_top, sx, sy, sw, sh, dx, dy, dw, dh)
    x -= cvs_left
    y -= cvs_top
    x = min(max(x, dx), dx + dw)
    y = min(max(y, dy), dy + dh)
    return [(x - dx) / dw * sw + sx, (y - dy) / dh * sh + sy]

In [7]:
img_root = '\outputVideo'  
fps = 60  

fourcc = cv2.VideoWriter_fourcc(*'XVID')
videoWriter = cv2.VideoWriter('outputVideo/TestVideo.mp4',fourcc,fps,(1920,1080),True)

In [8]:
pytesseract.pytesseract.tesseract_cmd = r'E:\Tesseract\tesseract.exe' # where your tesseract install
capture = cv2.VideoCapture('rawVideo/test1.mp4')
image_coords = []
valid_area = Polygon([(0, 50), (0, 1000), (1195, 50), (1195, 285), (1418, 285), (1418, 1000)])
if capture.isOpened():
    i = 0
    while True:
        success, frame = capture.read()
        i += 1
        if not success:
            break
        if frame is not None:
            gs_frame = cv2.GaussianBlur(frame, (5, 5), 0)
            hsv = cv2.cvtColor(gs_frame, cv2.COLOR_BGR2HSV)
            erode_hsv = cv2.erode(hsv, None, iterations=2)
            inRange_hsv = cv2.inRange(erode_hsv, color_dict[circle_color]['Lower'], color_dict[circle_color]['Upper'])
            
            # draw rect border
            contours = cv2.findContours(inRange_hsv.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
            if len(contours) != 0:
                c = max(contours, key=cv2.contourArea)
                rect = cv2.minAreaRect(c)
                box = cv2.boxPoints(rect)
                cv2.drawContours(frame, [np.int0(box)], -1, (0, 255, 255), 2)
                

                # find center
                M = cv2.moments(c)
                center_x = int(M['m10'] / M['m00'])
                center_y = int(M['m01'] / M['m00'])
                image_coords.append([center_x, center_y])
                # if not valid_area.contains(Point(center_x, center_y)):  
                #     continue
                frame_copy = frame.copy()
                cv2.circle(frame_copy, (center_x, center_y), 7, 128, -1)
                text = '(' + str(center_x) + ', ' + str(center_y) + ')'
                cv2.putText(frame_copy, text, (center_x - 50, center_y + 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
                # cv2.imshow("img", frame_copy)
                # cv2.waitKey()
                # cv2.destroyAllWindows()
                videoWriter.write(frame_copy)
videoWriter.release()
        